# 02 Difference-in-Differences + Event Study

TWFE DiD and event studies with synthetic adoption and diagnostics.


## Table of Contents
- [Synthetic adoption + treatment](#synthetic-adoption-treatment)
- [TWFE DiD](#twfe-did)
- [Event study (leads/lags)](#event-study-leads-lags)
- [Diagnostics: pre-trends + placebo](#diagnostics-pre-trends-placebo)
- [Checkpoint (Self-Check)](#checkpoint-self-check)
- [Solutions (Reference)](#solutions-reference)


## Why This Notebook Matters
Causal notebooks focus on **identification**: what would have to be true for a coefficient to represent a causal effect.
You will practice:
- building a county-year panel,
- fixed effects (TWFE),
- clustered standard errors,
- DiD + event studies,
- IV/2SLS.


## What You Will Produce
- (no file output; learning/analysis notebook)

## Success Criteria
- You can explain what you built and why each step exists.
- You can run your work end-to-end without undefined variables.

## Common Pitfalls
- Running cells top-to-bottom without reading the instructions.
- Leaving `...` placeholders in code cells.
- Treating regression output as causal without stating identification assumptions.
- Using non-clustered SE when shocks are correlated within groups (e.g., states).

## Matching Guide
- `docs/guides/07_causal/02_difference_in_differences_event_study.md`



## How To Use This Notebook
- This notebook is hands-on. Most code cells are incomplete on purpose.
- Complete each TODO, then run the cell.
- Use the matching guide (`docs/guides/07_causal/02_difference_in_differences_event_study.md`) for deep explanations and alternative examples.
- Write short interpretation notes as you go (what changed, why it matters).



<a id="environment-bootstrap"></a>
## Environment Bootstrap
Run this cell first. It makes the repo importable and defines common directories.



In [ ]:
from __future__ import annotations

from pathlib import Path
import sys


def find_repo_root(start: Path) -> Path:
    p = start
    for _ in range(8):
        if (p / 'src').exists() and (p / 'docs').exists():
            return p
        p = p.parent
    raise RuntimeError('Could not find repo root. Start Jupyter from the repo root.')


PROJECT_ROOT = find_repo_root(Path.cwd())
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

DATA_DIR = PROJECT_ROOT / 'data'
RAW_DIR = DATA_DIR / 'raw'
PROCESSED_DIR = DATA_DIR / 'processed'
SAMPLE_DIR = DATA_DIR / 'sample'

PROJECT_ROOT



## Goal
Practice DiD and event studies using:
- a real county-year outcome (poverty rate)
- a **synthetic**, deterministic adoption schedule by state
- a **semi-synthetic** outcome with a known injected treatment effect

This is a method exercise, not a real policy evaluation.



## Primer: Panel and IV regression with `linearmodels`

`statsmodels` is great for OLS inference, but panel and IV workflows are often cleaner with `linearmodels`.

This project uses `linearmodels` for:
- **PanelOLS** (fixed effects / TWFE)
- **IV2SLS** (instrumental variables)

### Panel data shape
Most panel estimators expect a **MultiIndex**:
- level 0: entity (e.g., county `fips`)
- level 1: time (e.g., `year`)

In pandas:
```python
# df is a DataFrame with columns fips, year, y, x1, x2
# df = df.set_index(['fips', 'year']).sort_index()
```

### Minimal PanelOLS (two-way fixed effects)
```python
from linearmodels.panel import PanelOLS

# y: Series with MultiIndex
# X: DataFrame with MultiIndex

# model: y_it = beta'X_it + alpha_i + gamma_t + eps_it
# res = PanelOLS(y, X, entity_effects=True, time_effects=True).fit(cov_type='robust')
# print(res.summary)
```

### Clustered standard errors (common in applied work)
If errors are correlated within clusters (e.g., state), use clustered SE:
```python
# clusters must align with y/X index (same rows)
# res = PanelOLS(y, X, entity_effects=True, time_effects=True).fit(
#     cov_type='clustered',
#     clusters=df['state'],  # e.g., state-level clustering
# )
```

### Minimal IV2SLS (one endogenous regressor)
```python
from linearmodels.iv import IV2SLS
import statsmodels.api as sm

# y: Series
# endog: DataFrame with endogenous regressor(s)
# exog: DataFrame with controls (include a constant)
# instr: DataFrame with instruments

# exog = sm.add_constant(exog, has_constant='add')
# res = IV2SLS(y, exog, endog, instr).fit(cov_type='robust')
# print(res.summary)
```

### Practical rule
- If the goal is **causal identification**, always write down the assumptions first (parallel trends, exclusion restriction, etc.).
- Then treat the model output as conditional on those assumptions, not as “truth”.


<a id="synthetic-adoption-treatment"></a>
## Synthetic adoption + treatment

### Goal
Define a deterministic adoption year by state and build:
- `treated_it`
- `poverty_rate_semi` (known post-treatment effect)



### Your Turn: Load panel and create synthetic adoption


In [ ]:
import numpy as np
import pandas as pd

path = PROCESSED_DIR / 'census_county_panel.csv'
if path.exists():
    df = pd.read_csv(path)
else:
    df = pd.read_csv(SAMPLE_DIR / 'census_county_panel_sample.csv')

df['fips'] = df['fips'].astype(str)
df['year'] = df['year'].astype(int)
df['state'] = df['state'].astype(str).str.zfill(2)

states = sorted(df['state'].unique())
# Deterministic adoption schedule (edit if you want):
adopt = {states[0]: 2018, states[1]: 2020}  # remaining states are never-treated

df['adopt_year'] = df['state'].map(adopt)
df['ever_treated'] = df['adopt_year'].notna().astype(int)
df['post'] = ((df['year'] >= df['adopt_year']).fillna(False)).astype(int)
df['treated'] = df['ever_treated'] * df['post']

true_effect = -0.02
df['poverty_rate_real'] = df['poverty_rate'].astype(float)
df['poverty_rate_semi'] = (df['poverty_rate_real'] + true_effect * df['treated']).clip(0, 1)

df[['state', 'year', 'treated', 'poverty_rate_real', 'poverty_rate_semi']].head()



<a id="twfe-did"></a>
## TWFE DiD

### Goal
Estimate the effect of treatment with TWFE DiD:
- county FE
- year FE
- clustered SE by state (common)



### Your Turn: Fit TWFE DiD


In [ ]:
from src.causal import fit_twfe_panel_ols

# Panel index
df = df.set_index(['fips', 'year'], drop=False).sort_index()

# TODO: Fit DiD on semi-synthetic outcome
res_did = fit_twfe_panel_ols(
    df,
    y_col='poverty_rate_semi',
    x_cols=['treated'],
    entity_effects=True,
    time_effects=True,
    cluster_col='state',
)

res_did.params



<a id="event-study-leads-lags"></a>
## Event study (leads/lags)

### Goal
Estimate dynamic effects around adoption and inspect pre-trends.



### Your Turn: Build leads/lags and fit


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df_es = df.reset_index(drop=True).copy()
df_es['event_time'] = df_es['year'] - df_es['adopt_year']

window = list(range(-3, 4))
base = -1
event_cols = []
for k in window:
    if k == base:
        continue
    col = f'event_{k}'
    df_es[col] = ((df_es['ever_treated'] == 1) & (df_es['event_time'] == k)).astype(int)
    event_cols.append(col)

df_es = df_es.set_index(['fips', 'year'], drop=False).sort_index()

res_es = fit_twfe_panel_ols(
    df_es,
    y_col='poverty_rate_semi',
    x_cols=event_cols,
    entity_effects=True,
    time_effects=True,
    cluster_col='state',
)

coefs = res_es.params.filter(like='event_')
ses = res_es.std_errors.filter(like='event_')
out = coefs.to_frame('coef').join(ses.to_frame('se'))
out['k'] = out.index.str.replace('event_', '').astype(int)
out = out.sort_values('k')

# TODO: Plot coefficient path with 95% CI
plt.errorbar(out['k'], out['coef'], yerr=1.96*out['se'], fmt='o-')
plt.axhline(0, color='gray', linestyle='--')
plt.axvline(base, color='gray', linestyle=':')
plt.xlabel('Event time (years relative to adoption)')
plt.ylabel('Effect')
plt.title('Event study (semi-synthetic)')
plt.show()



<a id="diagnostics-pre-trends-placebo"></a>
## Diagnostics: pre-trends + placebo

### Goal
Run at least one falsification / diagnostic.

Suggestions:
- Pre-trends: are lead coefficients near 0?
- Placebo: shift adoption years earlier for treated states.
- Re-run on the real outcome (`poverty_rate_real`) and reflect on why it is not causal.



### Your Turn: One diagnostic


In [ ]:
# TODO: Implement one diagnostic and summarize what you found.
...



<a id="checkpoint-self-check"></a>
## Checkpoint (Self-Check)
Run a few asserts and write 2-3 sentences summarizing what you verified.



In [ ]:
import pandas as pd

# Expected output: (see notebook front matter)
# TODO: If you created a panel DataFrame, verify the indexing + core columns.
# Example (adjust variable names):
# assert isinstance(panel.index, pd.MultiIndex)
# assert panel.index.names[:2] == ['fips', 'year']
# assert panel['year'].astype(int).between(1900, 2100).all()
# assert panel['fips'].astype(str).str.len().eq(5).all()
#
# TODO: Write 2-3 sentences:
# - What is the identification assumption for your causal estimate?
# - What diagnostic/falsification did you run?
...



## Extensions (Optional)
- Try one additional variant beyond the main path (different features, different split, different model).
- Write down what improved, what got worse, and your hypothesis for why.



## Reflection
- What did you assume implicitly (about timing, availability, stationarity, or costs)?
- If you had to ship this model, what would you monitor?



<a id="solutions-reference"></a>
## Solutions (Reference)

Try the TODOs first. Use these only to unblock yourself or to compare approaches.

<details><summary>Solution: Synthetic adoption + treatment</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 02_difference_in_differences_event_study — Synthetic adoption + treatment
import numpy as np
import pandas as pd

df = pd.read_csv(SAMPLE_DIR / 'census_county_panel_sample.csv')
df['fips'] = df['fips'].astype(str)
df['year'] = df['year'].astype(int)

states = sorted(df['state'].astype(str).unique())
# Deterministic synthetic adoption schedule
adopt = {states[0]: 2018, states[1]: 2020}  # states[2] is never-treated

df['adopt_year'] = df['state'].astype(str).map(adopt)
df['ever_treated'] = df['adopt_year'].notna().astype(int)
df['post'] = ((df['year'] >= df['adopt_year']).fillna(False)).astype(int)
df['treated'] = df['ever_treated'] * df['post']

# Semi-synthetic outcome: add a known post effect.
true_effect = -0.02
df['poverty_rate_real'] = df['poverty_rate'].astype(float)
df['poverty_rate_semi'] = (df['poverty_rate_real'] + true_effect * df['treated']).clip(0, 1)

df[['state', 'year', 'treated', 'poverty_rate_real', 'poverty_rate_semi']].head()
```

</details>

<details><summary>Solution: TWFE DiD</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 02_difference_in_differences_event_study — TWFE DiD
from src.causal import fit_twfe_panel_ols

df = df.set_index(['fips', 'year'], drop=False).sort_index()

res = fit_twfe_panel_ols(
    df,
    y_col='poverty_rate_semi',
    x_cols=['treated'],
    entity_effects=True,
    time_effects=True,
    cluster_col='state',
)
res.params
```

</details>

<details><summary>Solution: Event study (leads/lags)</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 02_difference_in_differences_event_study — Event study (leads/lags)
import numpy as np

df_es = df.reset_index(drop=True).copy()
df_es['event_time'] = df_es['year'] - df_es['adopt_year']

window = list(range(-3, 4))
base = -1
event_cols = []
for k in window:
    if k == base:
        continue
    col = f'event_{k}'
    df_es[col] = ((df_es['ever_treated'] == 1) & (df_es['event_time'] == k)).astype(int)
    event_cols.append(col)

df_es = df_es.set_index(['fips', 'year'], drop=False).sort_index()

res_es = fit_twfe_panel_ols(
    df_es,
    y_col='poverty_rate_semi',
    x_cols=event_cols,
    entity_effects=True,
    time_effects=True,
    cluster_col='state',
)

coefs = res_es.params.filter(like='event_')
ses = res_es.std_errors.filter(like='event_')
out = (coefs.to_frame('coef').join(ses.to_frame('se')))
out
```

</details>

<details><summary>Solution: Diagnostics: pre-trends + placebo</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 02_difference_in_differences_event_study — Diagnostics: pre-trends + placebo
# Pre-trends: inspect lead coefficients (event_-3, event_-2).
# Placebo: shift adoption earlier and confirm estimated effect shrinks toward 0.
```

</details>

